In [ ]:
pip install ipywidgets

In [58]:
import ipywidgets as widgets
from IPython.display import display
import os
import json

with open("quiz.json", "r", encoding="utf-8") as f:
    quiz = json.load(f)


def render_image(path, width=600):
    if not os.path.exists(path):
        return widgets.HTML(f"<i>Image not found: {path}</i>")

    with open(path, "rb") as f:
        data = f.read()

    return widgets.Image(
        value=data,
        format=path.split(".")[-1],
        width=width
    )

def render_images_vertical(paths, width=500):
    widgets_list = []
    for i, path in enumerate(paths, start=1):
        img = render_image(path, width)
        label = widgets.HTML(f"<b>Figure {i}</b>")
        widgets_list.append(widgets.VBox([label, img]))
    return widgets.VBox(widgets_list)

def render_question(q):
    title = widgets.HTML(f"<b>{q['question']}</b>")
    elements = [title]

    if "images" in q:
        elements.append(render_images_vertical(q["images"]))
    elif "image" in q:
        elements.append(render_image(q["image"]))
        
    button = widgets.Button(description="Verify Answer")
    output = widgets.Output()

    # single
    if q["type"] == "single":
        options = widgets.RadioButtons(
            options=[(text, i) for i, text in enumerate(q["options"])],
            layout={'width': 'max-content'}
        )

        def verify(b):
            with output:
                output.clear_output()
                if options.value is None:
                    print("Please select an option.")
                    return

                selected = options.value
                print(q["feedback"][str(selected)])

        button.on_click(verify)
        display(widgets.VBox(elements + [options, button, output]))

    # multi
    elif q["type"] == "multi":
        checkboxes = []
        for i, text in enumerate(q["options"]):
            cb = widgets.Checkbox(
                description=text,
                indent=False,
                style={'description_width': 'initial'},
                layout=widgets.Layout(
                    width='100%',
                    margin='0 0 0 0',
                    padding='0 0 0 0'
                )
            )
            checkboxes.append((i, cb))
    
        def verify(b):
            with output:
                output.clear_output()
                selected = {i for i, cb in checkboxes if cb.value}
    
                if selected == set(q["correct"]):
                    print("All correct\n")
                    for i in sorted(selected):
                        print(q["feedback"][str(i)])
                else:
                    print("One or more selected options are incorrect.")
    
        button.on_click(verify)
    
        display(widgets.VBox(
            elements +
            [cb for _, cb in checkboxes] +
            [button, output]
        ))


for q in quiz:
    render_question(q)